In [12]:
import os
import sys
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim,\
                             mean_squared_error as mse,\
                             normalized_root_mse as nrmse,\
                             normalized_mutual_information as nmi


In [13]:
# Find the ERO-SNN folder and add it to the python path
current_dir = os.getcwd()

while os.path.basename(current_dir) != 'ERO-SNN':
    print(os.path.basename(current_dir))
    current_dir = os.path.dirname(current_dir)
    
print(f"Found ERO-SNN folder: {current_dir}")
sys.path.append(current_dir)
os.chdir(current_dir)

Found ERO-SNN folder: /home/bernard/Desktop/IIT_Thesis/ERO-SNN


In [14]:
# Extracting the data
datasets = ['h36m', 'EyeTracking', 'MVSEC']
Number_of_Neighbours = [4, 8, 12, 20]
data_path = "Meshed Videos"
video_batches = {}
for dataset in datasets:
    video_batches[dataset] = {}
    for filename in os.listdir(data_path):
        if filename.startswith(dataset):
            print(f"Found {filename} in {dataset}")
            input_path = os.path.join(data_path, filename, "input-out.mp4")
            eros_path = os.path.join(data_path, filename, "eros-binary-out.mp4")
            for neighbours in Number_of_Neighbours:
                scm_path = os.path.join(data_path, filename, f"N{neighbours}", "scm-out.mp4")
                video_batches[dataset][neighbours] = [input_path, eros_path, scm_path] 
                print(f"Found {len(video_batches[dataset][neighbours])} files for {dataset} with {neighbours} neighbours")

Found h36m_sample in h36m
Found 3 files for h36m with 4 neighbours
Found 3 files for h36m with 8 neighbours
Found 3 files for h36m with 12 neighbours
Found 3 files for h36m with 20 neighbours
Found EyeTracking in EyeTracking
Found 3 files for EyeTracking with 4 neighbours
Found 3 files for EyeTracking with 8 neighbours
Found 3 files for EyeTracking with 12 neighbours
Found 3 files for EyeTracking with 20 neighbours
Found MVSEC_short_outdoor in MVSEC
Found 3 files for MVSEC with 4 neighbours
Found 3 files for MVSEC with 8 neighbours
Found 3 files for MVSEC with 12 neighbours
Found 3 files for MVSEC with 20 neighbours


In [15]:
results = {dataset: {neighbours:{} for neighbours in Number_of_Neighbours} for dataset in datasets}

generate_scores = {'ssim_scores': True, 'mse_scores': False, 'nmse_scores': False, 'nmi_scores': True}

for dataset in datasets:
    for neighbours in Number_of_Neighbours:
        eros_path = video_batches[dataset][neighbours][1]
        scm_path = video_batches[dataset][neighbours][2]
        input_path = video_batches[dataset][neighbours][0]
        
        print(f"Dataset: {dataset}, Neighbours: {neighbours}")
        print(f"Processing combination:\n Input: {input_path},\n EROSb: {eros_path},\n SCM: {scm_path}\n")

        # Load videos
        input_video = cv2.VideoCapture(input_path)
        eros_video = cv2.VideoCapture(eros_path)
        scm_video = cv2.VideoCapture(scm_path)

        # Check if the videos are the same length and frame rate
        input_fps = input_video.get(cv2.CAP_PROP_FPS)
        eros_fps = eros_video.get(cv2.CAP_PROP_FPS)
        scm_fps = scm_video.get(cv2.CAP_PROP_FPS)
        
        input_frames = input_video.get(cv2.CAP_PROP_FRAME_COUNT)
        eros_frames = eros_video.get(cv2.CAP_PROP_FRAME_COUNT)
        scm_frames = scm_video.get(cv2.CAP_PROP_FRAME_COUNT)

        print(f"Input FPS: {input_fps}, EROSb FPS: {eros_fps}, SCM FPS: {scm_fps}")
        print(f"Input Frames: {input_frames}, EROSb Frames: {eros_frames}, SCM Frames: {scm_frames}")

        ssim_scores = {'Input-EROSb': [], 'EROSb-SCM': [], 'Input-SCM': []}
        mse_scores = {'Input-EROSb': [], 'EROSb-SCM': [], 'Input-SCM': []}
        nmse_scores = {'Input-EROSb': [], 'EROSb-SCM': [], 'Input-SCM': []}
        nmi_scores = {'Input-EROSb': [], 'EROSb-SCM': [], 'Input-SCM': []}

        frame_index = 0
        while True:
            # Read frames from all videos
            ret1, frame1 = input_video.read()
            ret2, frame2 = eros_video.read()
            ret3, frame3 = scm_video.read()

            # Break the loop if any video ends
            if not ret1 or not ret2 or not ret3:
                break

            # Check if frames are in sync by comparing frame indices
            input_frame_idx = input_video.get(cv2.CAP_PROP_POS_FRAMES)
            eros_frame_idx = eros_video.get(cv2.CAP_PROP_POS_FRAMES)
            scm_frame_idx = scm_video.get(cv2.CAP_PROP_POS_FRAMES)

            if not (input_frame_idx == eros_frame_idx == scm_frame_idx == frame_index + 1):
                print(f"Error: Frames are out of sync at frame {frame_index}.")
                print(f"Input Frame Index: {input_frame_idx}, EROS Frame Index: {eros_frame_idx}, SCM Frame Index: {scm_frame_idx}")
                break

            frame_index += 1

            # Convert frames to grayscale
            gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            gray3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)
            
            # Apply binary threshold to convert to black and white
            _, bw1 = cv2.threshold(gray1, 254, 255, cv2.THRESH_BINARY)
            _, bw2 = cv2.threshold(gray2, 254, 255, cv2.THRESH_BINARY)
            _, bw3 = cv2.threshold(gray3, 254, 255, cv2.THRESH_BINARY)

            # Compute SSIM between the two frames and append to the list
            if generate_scores['ssim_scores']:
                ssim12, _ = ssim(bw1, bw2, full=True, data_range=255, sample_covariance=False, gaussian_weights=True, sigma=1.5)
                ssim23, _ = ssim(bw2, bw3, full=True, data_range=255, sample_covariance=False, gaussian_weights=True, sigma=1.5)
                ssim13, _ = ssim(bw1, bw3, full=True, data_range=255, sample_covariance=False, gaussian_weights=True, sigma=1.5)
                ssim_scores['Input-EROSb'].append(ssim12)
                ssim_scores['EROSb-SCM'].append(ssim23)            
                ssim_scores['Input-SCM'].append(ssim13)
            
            # Compute MSE between the frames and append to the list
            if generate_scores['mse_scores']:
                mse12 = mse(bw1, bw2)
                mse23 = mse(bw2, bw3)
                mse13 = mse(bw1, bw3)
                mse_scores['Input-EROSb'].append(mse12)
                mse_scores['EROSb-SCM'].append(mse23)            
                mse_scores['Input-SCM'].append(mse13)

            # Compute NMSE between the frames and append to the list
            if generate_scores['nmse_scores']:
                denom12 = np.sqrt(mse(bw1, bw1))
                denom23 = np.sqrt(mse(bw2, bw2))
                denom13 = np.sqrt(mse(bw3, bw3))
                
                if denom12 != 0:
                    nmse12 = nrmse(bw1, bw2)
                else:
                    nmse12 = float('inf')  # or some predefined value

                if denom23 != 0:
                    nmse23 = nrmse(bw2, bw3)
                else:
                    nmse23 = float('inf')  # or some predefined value

                if denom13 != 0:
                    nmse13 = nrmse(bw1, bw3)
                else:
                    nmse13 = float('inf')  # or some predefined value

                nmse_scores['Input-EROSb'].append(nmse12)
                nmse_scores['EROSb-SCM'].append(nmse23)
                nmse_scores['Input-SCM'].append(nmse13)
            
            # Compute NMI between the frames and append to the list
            if generate_scores['nmi_scores']:
                nmi12 = nmi(bw1, bw2)
                nmi23 = nmi(bw2, bw3)
                nmi13 = nmi(bw1, bw3)
                nmi_scores['Input-EROSb'].append(nmi12)
                nmi_scores['EROSb-SCM'].append(nmi23)
                nmi_scores['Input-SCM'].append(nmi13)
            
        # Release video captures
        input_video.release()
        eros_video.release()
        scm_video.release()
        
        # Create dataframes
        if generate_scores['ssim_scores']:
            results[dataset][neighbours]['ssim_scores'] = pd.DataFrame(ssim_scores)
        if generate_scores['mse_scores']:
            results[dataset][neighbours]['mse_scores'] = pd.DataFrame(mse_scores)
        if generate_scores['nmse_scores']:
            results[dataset][neighbours]['nmse_scores'] = pd.DataFrame(nmse_scores)
        if generate_scores['nmi_scores']:
            results[dataset][neighbours]['nmi_scores'] = pd.DataFrame(nmi_scores)

Dataset: h36m, Neighbours: 4
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROSb: Meshed Videos/h36m_sample/eros-binary-out.mp4,
 SCM: Meshed Videos/h36m_sample/N4/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 1076.0, EROSb Frames: 1076.0, SCM Frames: 1076.0
Dataset: h36m, Neighbours: 8
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROSb: Meshed Videos/h36m_sample/eros-binary-out.mp4,
 SCM: Meshed Videos/h36m_sample/N8/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 1076.0, EROSb Frames: 1076.0, SCM Frames: 1076.0
Dataset: h36m, Neighbours: 12
Processing combination:
 Input: Meshed Videos/h36m_sample/input-out.mp4,
 EROSb: Meshed Videos/h36m_sample/eros-binary-out.mp4,
 SCM: Meshed Videos/h36m_sample/N12/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 1076.0, EROSb Frames: 1076.0, SCM Frames: 1076.0
Dataset: h36m, Neighbours: 20
Pr

/home/bernard/.virtualenvs/IIT/lib/python3.10/site-packages/skimage/metrics/simple_metrics.py:270: RuntimeWarning: invalid value encountered in scalar divide
  return (H0 + H1) / H01


Dataset: EyeTracking, Neighbours: 8
Processing combination:
 Input: Meshed Videos/EyeTracking/input-out.mp4,
 EROSb: Meshed Videos/EyeTracking/eros-binary-out.mp4,
 SCM: Meshed Videos/EyeTracking/N8/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 9245.0, EROSb Frames: 9245.0, SCM Frames: 9244.0
Dataset: EyeTracking, Neighbours: 12
Processing combination:
 Input: Meshed Videos/EyeTracking/input-out.mp4,
 EROSb: Meshed Videos/EyeTracking/eros-binary-out.mp4,
 SCM: Meshed Videos/EyeTracking/N12/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 9245.0, EROSb Frames: 9245.0, SCM Frames: 9244.0
Dataset: EyeTracking, Neighbours: 20
Processing combination:
 Input: Meshed Videos/EyeTracking/input-out.mp4,
 EROSb: Meshed Videos/EyeTracking/eros-binary-out.mp4,
 SCM: Meshed Videos/EyeTracking/N20/scm-out.mp4

Input FPS: 33.333, EROSb FPS: 33.333, SCM FPS: 33.333
Input Frames: 9245.0, EROSb Frames: 9245.0, SCM Frames: 9244.0
Dataset: 

In [23]:
# Create a directory to save the results
output_dir = 'quantitative_results_binarized'
os.makedirs(output_dir, exist_ok=True)

# Iterate through the results dictionary and save each DataFrame to a CSV file
for dataset_name, neighbors_dict in results.items():
    for n_neighbors, scores_dict in neighbors_dict.items():
        for score_type, df in scores_dict.items():
            # Define the filename
            filename = f"{dataset_name}_NN_{n_neighbors}_{score_type}.csv"
            filepath = os.path.join(output_dir, filename)
            
            # Save the DataFrame to a CSV file
            df.to_csv(filepath, index=False)
            print(f"Saved {filepath}")

Saved quantitative_results_binarized/h36m_NN_4_ssim_scores.csv
Saved quantitative_results_binarized/h36m_NN_4_nmi_scores.csv
Saved quantitative_results_binarized/h36m_NN_8_ssim_scores.csv
Saved quantitative_results_binarized/h36m_NN_8_nmi_scores.csv
Saved quantitative_results_binarized/h36m_NN_12_ssim_scores.csv
Saved quantitative_results_binarized/h36m_NN_12_nmi_scores.csv
Saved quantitative_results_binarized/h36m_NN_20_ssim_scores.csv
Saved quantitative_results_binarized/h36m_NN_20_nmi_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_4_ssim_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_4_nmi_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_8_ssim_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_8_nmi_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_12_ssim_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_12_nmi_scores.csv
Saved quantitative_results_binarized/EyeTracking_NN_20_ssim_scores.csv
Saved 

In [24]:
# Directory where the CSV files are saved
output_dir = 'quantitative_results_binarized'

# List of score types
# score_types = ['ssim_scores', 'mse_scores', 'nmse_scores', 'nmi_scores']
# score_types = score_types[generate_scores.values()]

# DataFrame to store the statistics
stats_df = pd.DataFrame(columns=['Dataset', 'Neighbors', 'Score Type', 'Column', 'Mean', 'Std Dev', 'Min', 'Max'])

# Iterate through the CSV files in the directory
for filename in os.listdir(output_dir):
    if filename.endswith('scores.csv'):
        # Extract dataset name, number of neighbors, and score type from the filename
        parts = filename.split('_')
        dataset_name = parts[0]
        n_neighbors = parts[2]
        score_type = parts[3].split('.')[0]
        
        # Read the CSV file into a DataFrame
        filepath = os.path.join(output_dir, filename)
        df = pd.read_csv(filepath)
        
        # Compute statistics
        mean_values = df.mean()
        std_values = df.std()
        min_values = df.min()
        max_values = df.max()
        
        # Create a temporary DataFrame to hold the statistics for the current file
        temp_df = pd.DataFrame({
            'Dataset': dataset_name,
            'Neighbors': n_neighbors,
            'Score Type': score_type,
            'Column': df.columns,
            'Mean': mean_values.values,
            'Std Dev': std_values.values,
            'Min': min_values.values,
            'Max': max_values.values
        })
        
        # Concatenate the temporary DataFrame with the main stats_df
        stats_df = pd.concat([stats_df, temp_df], ignore_index=True)
        
        # Generate a plot for the DataFrame
        plt.figure(figsize=(10, 6))
        for column in df.columns:
            plt.plot(df[column], label=column)
        
        plt.title(f'{score_type} for {dataset_name} with {n_neighbors} neighbors')
        plt.xlabel('Frame')
        plt.ylabel(score_type)
        plt.legend()
        plt.grid(True)
        
        # Save the plot
        plot_filename = f"{dataset_name}_NN_{n_neighbors}_{score_type}.png"
        plot_filepath = os.path.join(output_dir, plot_filename)
        plt.savefig(plot_filepath)
        plt.close()
        
        print(f"Saved plot {plot_filepath}")

# Save the statistics DataFrame to a CSV file
stats_filepath = os.path.join(output_dir, 'statistics_summary_binarized.csv')
stats_df.to_csv(stats_filepath, index=False)
print(f"Saved statistics summary to {stats_filepath}")

/tmp/ipykernel_1724125/2953954356.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stats_df = pd.concat([stats_df, temp_df], ignore_index=True)


Saved plot quantitative_results_binarized/MVSEC_NN_8_nmi.png
Saved plot quantitative_results_binarized/EyeTracking_NN_4_ssim.png
Saved plot quantitative_results_binarized/EyeTracking_NN_20_nmi.png
Saved plot quantitative_results_binarized/MVSEC_NN_20_ssim.png
Saved plot quantitative_results_binarized/EyeTracking_NN_8_nmi.png
Saved plot quantitative_results_binarized/h36m_NN_4_nmi.png
Saved plot quantitative_results_binarized/MVSEC_NN_12_nmi.png
Saved plot quantitative_results_binarized/MVSEC_NN_12_ssim.png
Saved plot quantitative_results_binarized/h36m_NN_12_ssim.png
Saved plot quantitative_results_binarized/h36m_NN_20_nmi.png
Saved plot quantitative_results_binarized/EyeTracking_NN_8_ssim.png
Saved plot quantitative_results_binarized/MVSEC_NN_20_nmi.png
Saved plot quantitative_results_binarized/MVSEC_NN_4_nmi.png
Saved plot quantitative_results_binarized/EyeTracking_NN_12_ssim.png
Saved plot quantitative_results_binarized/h36m_NN_8_nmi.png
Saved plot quantitative_results_binarized/h36

In [25]:
output_dir = 'quantitative_results_binarized'
stat_sum = pd.read_csv(os.path.join(output_dir, 'statistics_summary_binarized.csv'))

# Drop nmse_scores
stat_sum = stat_sum[stat_sum['Score Type'] != 'nmse']
# display(stat_sum)

grouped = stat_sum.groupby(['Dataset', 'Neighbors', 'Score Type', 'Column']).mean()
display(grouped)

Mean   Std Dev       Min  \
Dataset     Neighbors Score Type Column                                      
EyeTracking 4         nmi        EROSb-SCM    1.015781  0.010206  1.000000   
                                 Input-EROSb  1.002139  0.008983  1.000000   
                                 Input-SCM    1.024787  0.069840  1.000000   
                      ssim       EROSb-SCM    0.844662  0.048450  0.530749   
                                 Input-EROSb  0.970296  0.043268  0.660501   
...                                                ...       ...       ...   
h36m        20        nmi        Input-EROSb  1.000405  0.000693  1.000000   
                                 Input-SCM    1.345098  0.093873  1.088294   
                      ssim       EROSb-SCM    0.941086  0.013854  0.900533   
                                 Input-EROSb  0.955268  0.014460  0.912731   
                                 Input-SCM    0.963994  0.006682  0.946532   

                                                   Max  
Dataset     Neighbors Score Type Column                 
EyeTracking 4         nmi        EROSb-SCM    1.127852  
                                 Input-EROSb  1.349812  
                                 Input-SCM    2.000000  
                      ssim       EROSb-SCM    1.000000  
                                 Input-EROSb  1.000000  
...                                                ...  
h36m        20        nmi        Input-EROSb  1.006526  
                                 Input-SCM    2.000000  
                      ssim       EROSb-SCM    0.999787  
                                 Input-EROSb  0.999787  
                                 Input-SCM    1.000000  

[72 rows x 4 columns]